In [3]:
from datetime import datetime, timedelta, timezone
from typing import Annotated

from fastapi import Depends, FastAPI, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
app = FastAPI()
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

from jose import JWTError, jwt

from passlib.context import CryptContext
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

from pydantic import BaseModel


In [ ]:

# to get a string like this run:
# openssl rand -hex 32
SECRET_KEY = "09d25e094faa6ca2556c818166b7a9563b93f7099f6f0f4caa6cf63b88e8d3e7"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE = timedelta(weeks=1)

HTTP_401 = HTTPException(
    401,
    "Could not validate jwt credentials",
)

class User(BaseModel):
    username: str
    hashed_password: str

fake_users_db = {
    "admin": {
        "username": "admin",
        "hashed_password": "$2b$12$EixZaYVK1fsbw1ZfbX3OXePaWxn96p36WQoeG6Lruj3vjPGga31lW",
    }
}

class Token(BaseModel):
    # Contains the jwt. Returned by /signup and /login
    access_token: str
    token_type: str

def sign_token(username: str) -> Token:
    return Token(
        access_token=jwt.encode(
            {
                "exp": datetime.now(timezone.utc) + ACCESS_TOKEN_EXPIRE,
                "sub": username,
            },
            SECRET_KEY,
            algorithm=ALGORITHM,
        ),
        token_type="bearer",
    )

async def require_user(token: str = Depends(oauth2_scheme)) -> User:
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        if username is None:
            raise HTTP_401
    except JWTError:
        raise HTTP_401
    try:
        user = User(**fake_users_db[username])
    except KeyError:
        raise HTTPException(404, detail="User not found")

class SignupForm(BaseModel):
    username: str
    password: str
    confirm_password: str

@app.post("/signup")
async def signup(form: SignupForm):
    if form.password != form.confirm_password:
        raise HTTPException(400, detail="Passwords do not match")
    if form.username in fake_users_db:
        raise HTTPException(400, detail="Username already taken")
    fake_users_db[form.username] = dict(
        username=form.username,
        hashed_password=pwd_context.hash(form.password),
    )
    return sign_token(form.username)

@app.post("/login")
async def login(form: OAuth2PasswordRequestForm = Depends()) -> Token:
    try:
        user = User(**fake_users_db[form.username])
    except KeyError:
        raise HTTP_401
    if not pwd_context.verify(form.password, user.hashed_password):
        raise HTTP_401
    return sign_token(user.username)

@app.get("/users/me/", response_model=User)
async def read_users_me(user: User = Depends(require_user)):
    return user
